# Import required Library 

In [181]:
from pandas_datareader import data
import pandas_datareader as pdr
import pandas as pd
import numpy as np
import talib as ta
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.gridspec as gridspec
from matplotlib.dates import date2num
#from matplotlib.finance import candlestick_ohlc as candlestick
from mplfinance.original_flavor import candlestick_ohlc as candlestick # pls install matplotlib for candle stick plot 
import datetime
import requests

# Market Data

## Download Market Data

In [182]:
#ticker = '5099.kl'
# Last 3 years closing prices starting from Jan 2, 2018.
 
#now = datetime.datetime.now()
#startyear = 2018
#startmonth = 1 
#startday = 1
#start = datetime.datetime(startyear, startmonth, startday)
#sec_id = pdr.get_data_yahoo(ticker, start, now)
#sec_id.tail()


## Download Crypto history data

In [183]:
ticker = 'ethbtc'
base = 'https://api.huobi.pro'
url = '/market/history/kline?period=15min&size=2000&symbol=ethusdt'
final_url = base + url

reqs = requests.get(final_url)

df = pd.DataFrame(reqs.json()['data'])
df["Date"] = pd.to_datetime(df["id"], unit='s', origin='unix')
df["Date"] = df["Date"].dt.strftime("%m/%d/%Y %H:%M:%S")
df.drop('id',axis = 1, inplace = True)
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date',inplace = True)
df.rename(columns={'high':'High','low':'Low','open':'Open','close':'Close','vol':'Volume'},inplace=True)
df['Adj Close']= df['Close']
#df = df[['High','Low','Open','Close','Volume','Adj Close']]
df.sort_index(inplace = True)

sec_id= df


# Create technical Indicator

In [184]:

#ticker = 'OPK'

# Download sample data
#sec_id = data.get_data_yahoo(ticker, '2014-06-01')

# Data for matplotlib finance plot
sec_id_ochl = np.array(pd.DataFrame({'0':date2num(sec_id.index.to_pydatetime()),
                                '1':sec_id.Open,
                                '2':sec_id.Close,
                                '3':sec_id.High,
                                '4':sec_id.Low}))

# Technical Analysis
SMA_FAST = 50
SMA_SLOW = 200
RSI_PERIOD = 14
RSI_AVG_PERIOD = 15
MACD_FAST = 12
MACD_SLOW = 26
MACD_SIGNAL = 9
STOCH_K = 14
STOCH_D = 3
SIGNAL_TOL = 3
Y_AXIS_SIZE = 12

analysis = pd.DataFrame(index = sec_id.index)

#analysis['sma_f'] = pd.rolling_mean(sec_id.Close, SMA_FAST)
analysis['sma_f'] = sec_id.Close.rolling(SMA_FAST).mean()

#analysis['sma_s'] = pd.rolling_mean(sec_id.Close, SMA_SLOW)
analysis['sma_s'] = sec_id.Close.rolling(SMA_SLOW).mean()
#analysis['rsi'] = ta.RSI(sec_id.Close.as_matrix(), RSI_PERIOD)
analysis['rsi'] = ta.RSI(sec_id.Close.to_numpy(), RSI_PERIOD)
#analysis['sma_r'] = pd.rolling_mean(analysis.rsi, RSI_AVG_PERIOD) # check shift
analysis['sma_r'] = analysis.rsi.rolling(RSI_AVG_PERIOD).mean() # check shift

#analysis['macd'], analysis['macdSignal'], analysis['macdHist'] = ta.MACD(sec_id.Close.as_matrix(), fastperiod=MACD_FAST, slowperiod=MACD_SLOW, signalperiod=MACD_SIGNAL)
analysis['macd'], analysis['macdSignal'], analysis['macdHist'] = ta.MACD(sec_id.Close.to_numpy(), fastperiod=MACD_FAST, slowperiod=MACD_SLOW, signalperiod=MACD_SIGNAL)
#analysis['stoch_k'], analysis['stoch_d'] = ta.STOCH(sec_id.High.as_matrix(), sec_id.Low.as_matrix(), sec_id.Close.as_matrix(), slowk_period=STOCH_K, slowd_period=STOCH_D)
analysis['stoch_k'], analysis['stoch_d'] = ta.STOCH(sec_id.High.to_numpy(), sec_id.Low.to_numpy(), sec_id.Close.to_numpy(), slowk_period=STOCH_K, slowd_period=STOCH_D)

analysis['sma'] = np.where(analysis.sma_f > analysis.sma_s, 1, 0)
analysis['macd_test'] = np.where((analysis.macd > analysis.macdSignal), 1, 0)
analysis['stoch_k_test'] = np.where((analysis.stoch_k < 50) & (analysis.stoch_k > analysis.stoch_k.shift(1)), 1, 0)
analysis['rsi_test'] = np.where((analysis.rsi < 50) & (analysis.rsi > analysis.rsi.shift(1)), 1, 0)
analysis[['Open','Close','High','Low']]= sec_id[['Open','Close','High','Low']]
# Make Shift for easy know cross line from nagetive 
analysis['macdHist_shift'] = analysis['macdHist'].shift(1)

# Prepare plot
fig, (ax5, ax3, ax4, ax2,ax1) = plt.subplots(5, 1, sharex=True)
ax1.set_ylabel(ticker, size=20)

#size plot
fig.set_size_inches(15,30)

# Plot candles
candlestick(ax1, sec_id_ochl, width=0.5, colorup='#53c156', colordown='#ff1717', alpha=1)

# Draw Moving Averages
analysis.sma_f.plot(ax=ax1, c='r')
analysis.sma_s.plot(ax=ax1, c='g')

#RSI
ax2.set_ylabel('RSI', size=Y_AXIS_SIZE)
analysis.rsi.plot(ax = ax2, c='g', label = 'Period: ' + str(RSI_PERIOD))
analysis.sma_r.plot(ax = ax2, c='r', label = 'MA: ' + str(RSI_AVG_PERIOD))
ax2.axhline(y=30, c='b')
ax2.axhline(y=50, c='black')
ax2.axhline(y=70, c='b')
ax2.set_ylim([0,100])
handles, labels = ax2.get_legend_handles_labels()
ax2.legend(handles, labels)

# Draw MACD computed with Talib
ax3.set_ylabel('MACD: '+ str(MACD_FAST) + ', ' + str(MACD_SLOW) + ', ' + str(MACD_SIGNAL), size=Y_AXIS_SIZE)
analysis.macd.plot(ax=ax3, color='b', label='Macd')
analysis.macdSignal.plot(ax=ax3, color='g', label='Signal')
analysis.macdHist.plot(ax=ax3, color='r', label='Hist')
ax3.axhline(0, lw=2, color='0')
handles, labels = ax3.get_legend_handles_labels()
ax3.legend(handles, labels)

# Stochastic plot
ax4.set_ylabel('Stoch (k,d)', size=Y_AXIS_SIZE)
analysis.stoch_k.plot(ax=ax4, label='stoch_k:'+ str(STOCH_K), color='r')
analysis.stoch_d.plot(ax=ax4, label='stoch_d:'+ str(STOCH_D), color='g')
handles, labels = ax4.get_legend_handles_labels()
ax4.legend(handles, labels)
ax4.axhline(y=20, c='b')
ax4.axhline(y=50, c='black')
ax4.axhline(y=80, c='b')


#Closing price 
ax5.set_ylabel('Closing price ', size=Y_AXIS_SIZE)
analysis['Close'].plot(ax=ax5, label='stoch_k:'+ str(STOCH_K), color='r')
analysis.sma_f.plot(ax=ax5, c='cyan')
analysis.sma_s.plot(ax=ax5, c='g')
#handles, labels = ax4.get_legend_handles_labels()
#ax5.legend(handles, labels)
#ax4.axhline(y=20, c='b')
#ax4.axhline(y=50, c='black')
#ax4.axhline(y=80, c='b')
plt.show()


# Buy and Sell position

## Buy Position

In [ ]:
buysell_record = pd.read_csv('Buysell record.csv')

buysell_record

,Index,Buy_Price,Target,Stop Loss,Status,Gain,Invest(usdt)
0,1,0,0,0,close,0,4000


In [ ]:
# call module ' load market price/data' 
# for loop, read data line by line 
for x in range (len(analysis)):
    #print (buysell_record['Status'].iloc[-1])
    if buysell_record['Status'].iloc[-1] == 'close':
        #print('Status is CLOSE')
        # call buy module 
        if (analysis['macdHist_shift'][x] <  0) and (analysis['macdHist'][x] >  0) and (analysis['macd'][x] < -0.02):
            #print ('I am in buy section')
            buy_price = analysis['Close'][x]
            Target_gain = 1.01
            StopLoss_rate = 1 - 0.01
            TargetSellPrice = buy_price * Target_gain
            Stoploss_= buy_price * StopLoss_rate
            #print('Target_gain',Target_gain)
            #print('StopLoss_rate',StopLoss_rate)

            #print('Buy price === ', buy_price)
            #print('TargetSellPrice   ',TargetSellPrice)
            #print('Stoploss_ = ',Stoploss_)
            a_series = pd.Series([analysis.index[x],buy_price,TargetSellPrice,Stoploss_,'open',0,np.nan],index = buysell_record.columns)
            buysell_record= buysell_record.append(a_series,ignore_index=True)
                  
                
    elif buysell_record['Status'].iloc[-1] == 'open':
        #print('I am in Sell section ****')
        # call sell module 
        if (analysis['Close'][x] >= buysell_record['Target'].iloc[-1]) :
            buysell_record['Status'].iloc[-1]= 'close'

            buysell_record['Gain'].iloc[-1] = analysis['Close'][x]- buysell_record['Buy_Price'].iloc[-1]
            
        elif (analysis['Close'][x] < buysell_record['Stop Loss'].iloc[-1]):
            buysell_record['Status'].iloc[-1]= 'close'

            buysell_record['Gain'].iloc[-1] = analysis['Close'][x]- buysell_record['Buy_Price'].iloc[-1]



/Users/khawchanyong/opt/anaconda3/envs/myproject-env/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
buysell_record['Gain(%)']=(buysell_record['Gain']/ buysell_record['Buy_Price']) *100
for i in range(len(buysell_record)-1):
    buysell_record['Invest(usdt)'][i+1] = buysell_record['Invest(usdt)'][i] + ( buysell_record['Invest(usdt)'][i] * (buysell_record['Gain(%)'][i+1] /100))
buysell_record

<ipython-input-167-b7a08c7c2e33>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buysell_record['Invest(usdt)'][i+1] = buysell_record['Invest(usdt)'][i] + ( buysell_record['Invest(usdt)'][i] * (buysell_record['Gain(%)'][i+1] /100))


,Index,Buy_Price,Target,Stop Loss,Status,Gain,Invest(usdt),Gain(%)
0,1,0.00,0.0000,0.0000,close,0.00,4000.000000,NaN
1,2021-11-14 05:45:00,4642.12,4688.5412,4595.6988,close,-62.81,3945.878176,-1.353046
2,2021-11-14 10:15:00,4613.98,4660.1198,4567.8402,close,-50.04,3903.083941,-1.084530
3,2021-11-14 18:45:00,4547.35,4592.8235,4501.8765,close,52.63,3948.257352,1.157377
4,2021-11-15 19:30:00,4602.14,4648.1614,4556.1186,close,-53.98,3901.946955,-1.172933
5,2021-11-16 04:30:00,4347.59,4391.0659,4304.1141,close,-56.62,3851.130701,-1.302331
6,2021-11-16 11:15:00,4263.36,4305.9936,4220.7264,close,52.51,3898.563447,1.231658
7,2021-11-16 21:45:00,4254.40,4296.9440,4211.8560,close,-46.05,3856.365053,-1.082409
8,2021-11-17 04:45:00,4141.87,4183.2887,4100.4513,close,55.36,3907.909010,1.336594
9,2021-11-18 10:45:00,4239.14,4281.5314,4196.7486,close,-49.37,3862.396603,-1.164623


In [ ]:
buysell_record['Gain'].sum()

259.8299999999981

In [ ]:
buysell_record['Gain(%)'].sum()

7.063069062430353

In [ ]:
buysell_record['Invest(usdt)'].iloc[-1]

4275.7394917311485

In [ ]:
(buysell_record['Invest(usdt)'].iloc[-1] - buysell_record['Invest(usdt)'][0] ) * 4

1102.957966924594

In [ ]:

cond_buy = [
    (analysis['macdHist_shift'] <  0) & (analysis['macdHist'] >  0) & (analysis['macd'] < -0.02),
    ]
values_buy = [analysis['Close']]
values_BuySignal = ['Buy']

analysis['Buy_Price'] = np.select(cond_buy, values_buy,np.nan)
analysis['BuySell_Signal'] = np.select(cond_buy, values_BuySignal,' ')


## Sell Position

In [ ]:
TargetSell = np.nan
StopLoss = np.nan
analysis['Sell_Price'] = np.nan
for i in analysis.index:
    if analysis['Buy_Price'][i] > 0:
        BuyPrice = analysis['Buy_Price'][i]
        print('Buy Price  ++ ',BuyPrice)
        TargetSell = BuyPrice * 1.03
        StopLoss = BuyPrice * -1.04
        print('Target profit 2%', TargetSell)
    print('Close Price == ', analysis['Close'][i])
    if (analysis['Close'][i] > TargetSell) & (TargetSell != np.nan):
        SellPrice = analysis['Close'][i]
        print('Sell Price *** ', SellPrice)
        TargetSell = np.nan
        analysis['Sell_Price'][i] = analysis['Close'][i]
        analysis['BuySell_Signal'][i] = 'Sell'
    elif (analysis['Close'][i] < StopLoss) & (TargetSell != np.nan):
        SellPrice = analysis['Close'][i]
        analysis['BuySell_Signal'][i] = 'Sell'
        print('Sell Price losssssss *** ', SellPrice)
        TargetSell = np.nan
        #analysis['Buy_signal'][i] = analysis['Close'][i]
        

Close Price ==  4641.29
Close Price ==  4644.14
Close Price ==  4643.76
Close Price ==  4647.07
Close Price ==  4641.24
Close Price ==  4646.98
Close Price ==  4659.98
Close Price ==  4653.35
Close Price ==  4657.96
Close Price ==  4646.91
Close Price ==  4655.65
Close Price ==  4655.93
Close Price ==  4651.69
Close Price ==  4656.74
Close Price ==  4638.68
Close Price ==  4648.62
Close Price ==  4641.56
Close Price ==  4639.77
Close Price ==  4643.46
Close Price ==  4650.45
Close Price ==  4687.93
Close Price ==  4680.52
Close Price ==  4675.09
Close Price ==  4670.46
Close Price ==  4664.57
Close Price ==  4652.05
Close Price ==  4660.47
Close Price ==  4663.6
Close Price ==  4657.13
Close Price ==  4642.99
Close Price ==  4635.22
Close Price ==  4627.24
Close Price ==  4632.28
Close Price ==  4630.7
Close Price ==  4628.4
Close Price ==  4634.2
Close Price ==  4640.61
Close Price ==  4639.3
Close Price ==  4633.37
Close Price ==  4644.08
Close Price ==  4633.04
Close Price ==  4639.

<ipython-input-173-da1f0baf8f1b>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  analysis['Sell_Price'][i] = analysis['Close'][i]
<ipython-input-173-da1f0baf8f1b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  analysis['BuySell_Signal'][i] = 'Sell'


Close Price ==  4304.23
Close Price ==  4281.5
Close Price ==  4286.21
Close Price ==  4269.28
Close Price ==  4314.35
Close Price ==  4328.1
Close Price ==  4324.97
Close Price ==  4351.84
Close Price ==  4320.62
Close Price ==  4341.5
Close Price ==  4327.29
Close Price ==  4320.45
Close Price ==  4314.79
Close Price ==  4304.55
Close Price ==  4302.78
Close Price ==  4320.73
Close Price ==  4324.04
Close Price ==  4321.89
Close Price ==  4333.24
Close Price ==  4355.82
Close Price ==  4349.78
Close Price ==  4359.35
Close Price ==  4372.83
Close Price ==  4382.64
Close Price ==  4360.55
Close Price ==  4351.3
Close Price ==  4347.05
Close Price ==  4350.06
Close Price ==  4342.84
Close Price ==  4344.59
Close Price ==  4341.83
Close Price ==  4333.51
Close Price ==  4343.08
Close Price ==  4346.88
Close Price ==  4347.33
Close Price ==  4339.19
Close Price ==  4357.57
Close Price ==  4350.92
Close Price ==  4362.35
Close Price ==  4350.06
Close Price ==  4348.71
Close Price ==  4318

In [ ]:
#analysis ['BuySell_signal'] = np.where(((analysis['macdHist_shift'] <  0) & (analysis['macdHist'] >  0)), 'Buy',(np.where(((analysis['macdHist_shift'] >  0) & (analysis['macdHist'] <  0)),'Sell',np.nan)))

In [ ]:
#df_buy = analysis[(analysis['BuySell_signal'] == 'Buy') | (analysis['BuySell_signal'] == 'Sell')]
#df_buy['BuySell_signal'][0]== "Buy"
#
#df_buy['BuySell_signal'][0]== "Buy"

# Calculate Profit

In [ ]:
df_buysell = analysis[(analysis['BuySell_signal'] == 'Buy') | (analysis['BuySell_signal'] == 'Sell')]
if df_buysell['BuySell_signal'][0]== "Sell" :
    df_buysell.drop(df_buysell.index[0] ,axis= 0 , inplace = True)
df_buysell_pivot = df_buysell[['Close','BuySell_signal']].reset_index()

KeyError: 'BuySell_signal'

In [ ]:
df_buysell_pivot['Profit']= np.nan
for x in range(0,len(df_buysell_pivot),2):
    df_buysell_pivot['Profit'][x+1] = df_buysell_pivot['Close'][x+1] - df_buysell_pivot['Close'][x] 

<ipython-input-98-fe2734e7bdb3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_buysell_pivot['Profit'][x+1] = df_buysell_pivot['Close'][x+1] - df_buysell_pivot['Close'][x]


In [ ]:
df_buysell_pivot['Profit'].sum()

-0.1100001335144043

In [ ]:
df_buysell_pivot

,Date,Close,BuySell_signal,Profit
0,2021-03-22,2.75,Buy,NaN
1,2021-04-19,2.73,Sell,-0.02
2,2021-05-24,2.08,Buy,NaN
3,2021-07-12,2.20,Sell,0.12
4,2021-07-27,2.27,Buy,NaN
5,2021-07-28,2.12,Sell,-0.15
6,2021-09-06,2.14,Buy,NaN
7,2021-09-30,2.06,Sell,-0.08
8,2021-10-20,2.07,Buy,NaN
9,2021-11-11,2.09,Sell,0.02
